In [43]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import re

In [18]:
# Load IMDB dataset
max_features = 10000
maxlen = 200
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)


In [19]:
# Pad sequences
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)


In [21]:
# Build LSTM model
embedding_dim = 128
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))
optimizer = Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [22]:
# Train the model (you can adjust epochs and batch_size as needed)
model.fit(X_train, y_train, epochs=1, batch_size=128, validation_split=0.2)

157/157 [==============================] - 122s 747ms/step - loss: 0.4420 - accuracy: 0.7884 - val_loss: 0.3210 - val_accuracy: 0.8662


In [25]:
ypred_prob = model.predict(X_test)
ypred = (ypred_prob > 0.5).astype(int)

782/782 [==============================] - 51s 65ms/step


In [26]:
accuracy = accuracy_score(y_test, ypred)
print(f'Accuracy on test set: {accuracy}')

Accuracy on test set: 0.86308


In [27]:
# Example of predicting a review
def predict_review_sentiment(review):
    review_sequence = imdb.get_word_index()
    word_sequence = review.split()
    sequence = [review_sequence[word] if word in review_sequence and review_sequence[word] < max_features else 0 for word in word_sequence]
    padded_sequence = pad_sequences([sequence], maxlen=maxlen)
    prediction = model.predict_classes(padded_sequence)
    return "Positive" if prediction[0][0] == 1 else "Negative"

In [32]:
def classify_review(review, model, word_index, maxlen=200):
    # Preprocess the review
    review = review.lower()
    review = re.sub(r"[^a-zA-Z0-9\s]", "", review)
    words = review.split()
    sequences = [word_index[word] if word in word_index and word_index[word] < max_features else 0 for word in words]
    padded_sequence = pad_sequences([sequences], maxlen=maxlen)
    
    # Predict sentiment
    prediction = model.predict(padded_sequence)[0][0]
    
    # Return sentiment label
    return "Positive" if prediction > 0.5 else "Negative"

In [34]:
def classify_review_interactive():
    while True:
        review = input("Enter a review (type 'exit' to quit): ")
        if review.lower() == 'exit':
            break
        sentiment = classify_review(review, model, word_index)
        print(f'Predicted Sentiment: {sentiment}')


In [39]:
word_index = imdb.get_word_index()

In [46]:
# Interactive classification
classify_review_interactive()

1/1 [==============================] - 0s 24ms/step
Predicted Sentiment: Positive
1/1 [==============================] - 0s 28ms/step
Predicted Sentiment: Positive
1/1 [==============================] - 0s 24ms/step
Predicted Sentiment: Positive
1/1 [==============================] - 0s 27ms/step
Predicted Sentiment: Positive
1/1 [==============================] - 0s 27ms/step
Predicted Sentiment: Positive
1/1 [==============================] - 0s 21ms/step
Predicted Sentiment: Positive
1/1 [==============================] - 0s 36ms/step
Predicted Sentiment: Positive
1/1 [==============================] - 0s 24ms/step
Predicted Sentiment: Positive
1/1 [==============================] - 0s 28ms/step
Predicted Sentiment: Positive
1/1 [==============================] - 0s 27ms/step
Predicted Sentiment: Positive
1/1 [==============================] - 0s 27ms/step
Predicted Sentiment: Positive
1/1 [==============================] - 0s 26ms/step
Predicted Sentiment: Positive
1/1 [===========

In [ ]:
review = "The movie was bad but the actors were doing great acting"
sentiment = classify_review(model, review, word_index)
print(f'Review: {review}')
print(f'Sentiment: {sentiment}')